In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import svm
import tensorflow.keras.models as km
import tensorflow.keras.applications as ka
import tensorflow.keras.layers as kl


In [3]:
train = pd.read_hdf("train.hdf5")
valid = pd.read_hdf("validation.hdf5")
train = np.array(train)
valid = np.array(valid)

In [4]:
c_train = np.repeat(train[0,:],3)
c_train = c_train.reshape((train.shape[1],3))
c_train = c_train.reshape((1024,60,3))
c_train.shape

(1024, 60, 3)

In [5]:
c_valid = np.repeat(valid[0,:],3)
c_valid = c_valid.reshape((valid.shape[1],3))
c_valid = c_valid.reshape((1024,60,3))
c_valid.shape

(1024, 60, 3)

In [6]:
train_n = np.zeros((train.shape[0],1024,60,3))
train_n[1].shape
valid_n = np.zeros((valid.shape[0],1024,60,3))
valid_n[1].shape

(1024, 60, 3)

In [7]:
train = np.array(train)
train_n = np.zeros((train.shape[0],1024,60,3))
for i in range(train.shape[0]):
    r = np.repeat(train[i,:],3)
    r = r.reshape((train.shape[1],3))
    r = r.reshape((1024,60,3))
    train_n[i] = np.copy(r)
train_n.shape


(1677, 1024, 60, 3)

In [8]:
valid = np.array(valid)
valid_n = np.zeros((valid.shape[0],1024,60,3))
for i in range(valid.shape[0]):
    t = np.repeat(valid[i,:],3)
    t = t.reshape((valid.shape[1],3))
    t = t.reshape((1024,60,3))
    valid_n[i] = np.copy(t)
valid_n.shape

(594, 1024, 60, 3)

In [9]:
#from keras.applications import VGG16
#conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(1024, 60, 3))
batch_size = 20
model_VGG16_without_top = ka.VGG16(include_top=False, weights='imagenet')
conv_base = model_VGG16_without_top.predict(train_n)
conv_test = model_VGG16_without_top.predict(valid_n)

model = km.Sequential()
model.add(kl.Flatten())
model.add(kl.Dense(256, activation='relu'))
# the 3 corresponds to the three output classes
model.add(kl.Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
X = model.predict(conv_base)
X_test = model.predict(conv_test)

58892288/58889256 [==============================] - 5s 0us/step


In [17]:
print("[INFO] preparing the network to extract flattened inputs(X,X_test) for scoring..... done!!!")
import matplotlib.pyplot as plt
from sklearn import svm
# fit the model
clf = svm.OneClassSVM(nu=0.01, kernel="rbf", gamma='auto')
clf.fit(X)
y_pred_train = clf.predict(X)
y_pred_test = clf.predict(X_test)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
print(n_error_train)
print(n_error_test)


[INFO] preparing the network to extract flattened inputs(X,X_test) for scoring..... done!!!
18
95


In [13]:
y_pred_test[y_pred_test==1] = 0
y_pred_test[y_pred_test==-1] = 1
y_pred_test


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
idex = np.arange(valid.shape[0])
dic = {"seqID" : idex, "anomaly" : y_pred_test}
res = pd.DataFrame(dic)
res.to_csv("y_validation_template.csv",index = False,sep = ";")